In [1]:
import optuna
import warnings
import gc
import os
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datetime import datetime
from glob import glob
from tqdm import tqdm

warnings.filterwarnings(action = 'ignore')

In [2]:
train_2017 = []
train_2018 = []
train_2019 = []
train_2020 = []

for i, path in enumerate(sorted(glob("Data/train/KNOW*.csv"))):
    if i == 0:
        train_2017 = pd.read_csv(path)
        pass
    elif i==1:
        train_2018 = pd.read_csv(path)
        pass
    elif i==2:
        train_2019 = pd.read_csv(path)
        pass
    else:
        train_2020 = pd.read_csv(path)
        pass
    
test_2017 = []
test_2018 = []
test_2019 = []
test_2020 = []

for i, path in enumerate(sorted(glob("Data/test/KNOW*.csv"))):
    if i == 0:
        test_2017 = pd.read_csv(path)
        pass
    elif i==1:
        test_2018 = pd.read_csv(path)
        pass
    elif i==2:
        test_2019 = pd.read_csv(path)
        pass
    else:
        test_2020 = pd.read_csv(path)
        pass

### 2019년 데이터 결측치 처리

- 중앙값으로 결측치 처리
- data leakage 고려하여 train 데이터로 결측치 처리
- bq 컬럼에만 결측치 존재

In [3]:
for col in train_2019:
        train_2019[col].replace(' ', np.nan, inplace = True)
        if col != 'knowcode':
            test_2019[col].replace(' ', np.nan, inplace = True)

In [4]:
train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2019.loc[(train_2019['bq4'] == 2) & (train_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2019.loc[(test_2019['bq4'] == 2) & (test_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

In [5]:
train_2019.loc[(train_2019['bq4'] == 1) & (train_2019['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
test_2019.loc[(test_2019['bq4'] == 1) & (test_2019['bq4_1a'].isnull()), 'bq4_1a'] = '없음'

In [6]:
train_2019.loc[(train_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2019.loc[(train_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2019.loc[(test_2019['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2019.loc[(test_2019['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

In [7]:
train_2019.loc[(train_2019['bq5'] == 2) & (train_2019['bq5_1'].isnull()),'bq5_1'] = 0
test_2019.loc[(test_2019['bq5'] == 2) & (test_2019['bq5_1'].isnull()),'bq5_1'] = 0

In [8]:
train_2019.loc[(train_2019['bq5'] == 2) & (train_2019['bq5_2'].isnull()),'bq5_2'] = '없음'
test_2019.loc[(test_2019['bq5'] == 2) & (test_2019['bq5_2'].isnull()),'bq5_2'] = '없음'

In [9]:
train_2019.loc[train_2019['bq5_2'].isnull(), 'bq5_2'] = '없음'
test_2019.loc[test_2019['bq5_2'].isnull(), 'bq5_2'] = '없음'

In [10]:
train_2019.loc[train_2019['bq27_1'].isnull(), 'bq27_1'] = '없음'
test_2019.loc[test_2019['bq27_1'].isnull(), 'bq27_1'] = '없음'

In [11]:
train_2019['bq28_1'].replace('없음', np.nan, inplace = True)
test_2019['bq28_1'].replace('없음', np.nan, inplace = True)
train_2019['bq28_2'].replace('없음', np.nan, inplace = True)
test_2019['bq28_2'].replace('없음', np.nan, inplace = True)

In [12]:
train_2019['bq28_1'].replace(np.nan, 0,inplace = True)
test_2019['bq28_1'].replace(np.nan, 0,inplace = True)
train_2019['bq28_2'].replace(np.nan,0, inplace = True)
test_2019['bq28_2'].replace(np.nan, 0,inplace = True)

In [13]:
train_2019['bq29'].replace(np.nan,0, inplace = True)
test_2019['bq29'].replace(np.nan, 0,inplace = True)

In [14]:
total_2019 = train_2019
bq1_list = total_2019.bq1.unique()

In [15]:
for bq1 in bq1_list:
    mean_bq31_1 = round(total_2019.loc[(total_2019.bq1 == bq1) & (total_2019.bq31_1.notnull())].bq31_1.apply(lambda x: int(x)).mean())
    train_2019.loc[(train_2019.bq1 == bq1) & (train_2019.bq31_1.isnull()), 'bq31_1'] = mean_bq31_1
    test_2019.loc[(test_2019.bq1 == bq1) & (test_2019.bq31_1.isnull()), 'bq31_1'] = mean_bq31_1
    
    mean_bq31_2 = round(total_2019.loc[(total_2019.bq1 == bq1) & (total_2019.bq31_2.notnull())].bq31_2.apply(lambda x: int(x)).mean())
    
    try:
        mean_bq31_3 = round(total_2019.loc[(total_2019.bq1 == bq1) & (total_2019.bq31_3.notnull())].bq31_3.apply(lambda x: int(x)).mean())
    except:
        mean_bq31_3 = mean_bq31_2
    
    train_2019.loc[(train_2019.bq1 == bq1) & (train_2019.bq31_2.isnull()), 'bq31_2'] = mean_bq31_2
    test_2019.loc[(test_2019.bq1 == bq1) & (test_2019.bq31_2.isnull()), 'bq31_2'] = mean_bq31_2
    
    train_2019.loc[(train_2019.bq1 == bq1) & (train_2019.bq31_3.isnull()), 'bq31_3'] = mean_bq31_3
    test_2019.loc[(test_2019.bq1 == bq1) & (test_2019.bq31_3.isnull()), 'bq31_3'] = mean_bq31_3

In [16]:
train_2019[['bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24']] = train_2019[['bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24']].fillna('없다')
test_2019[['bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24']] = test_2019[['bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24']].fillna('없다')

In [17]:
train_2019['bq30'] = train_2019['bq30'].fillna(int(train_2019['bq30'].mode()))
test_2019['bq30'] = test_2019['bq30'].fillna(int(train_2019['bq30'].mode()))

In [18]:
for col in (train_2019.isnull().sum().loc[train_2019.isnull().sum() > 0]).index:
    train_2019[col] = train_2019[col].fillna(int(train_2019[col].mode()))
    test_2019[col] = test_2019[col].fillna(int(train_2019[col].mode()))    

In [19]:
test_2019.loc[test_2019['bq31_3'].isnull(), 'bq31_3'] = int(train_2019['bq31_3'].mode())

In [20]:
train_2019.loc[train_2019['bq6'].str.contains('[가-힣]', na = False), ['bq6']] = int(train_2019['bq6'].mode())
train_2019.loc[train_2019['bq7'].str.contains('[가-힣]', na = False), ['bq7']] = int(train_2019['bq7'].mode())
train_2019.loc[train_2019['bq8_3'].str.contains('[가-힣]', na = False), ['bq8_3']] = int(train_2019['bq8_3'].mode())
train_2019.loc[train_2019['bq9'].str.contains('[가-힣]', na = False), ['bq9']] = int(train_2019['bq9'].mode())
train_2019.loc[train_2019['bq10'].str.contains('[가-힣]', na = False), ['bq10']] = int(train_2019['bq10'].mode())
train_2019.loc[train_2019['bq11'].str.contains('[가-힣]', na = False), ['bq11']] = int(train_2019['bq11'].mode())
train_2019.loc[train_2019['bq12_4'].str.contains('[가-힣]', na = False), ['bq12_4']] = int(train_2019['bq12_4'].mode())

train_2019.loc[train_2019['bq20'].str.contains('[가-힣]', na = False), ['bq20']] = int(train_2019['bq20'].mode())
test_2019.loc[test_2019['bq20'].str.contains('[가-힣]', na = False), ['bq20']] = int(train_2019['bq20'].mode())

train_2019.loc[train_2019['bq21_3'].str.contains('[가-힣]', na = False), ['bq21_3']] = int(train_2019['bq21_3'].mode())
test_2019.loc[test_2019['bq21_3'].str.contains('[가-힣]', na = False), ['bq21_3']] = int(train_2019['bq21_3'].mode())

train_2019.loc[train_2019['bq25'].str.contains('[가-힣]', na = False), ['bq25']] = int(train_2019['bq25'].mode())

train_2019.loc[train_2019['bq27'].str.contains('[가-힣]', na = False), ['bq27']] = int(train_2019['bq27'].mode())
test_2019.loc[test_2019['bq27'].str.contains('[가-힣]', na = False), ['bq27']] = int(train_2019['bq27'].mode())

test_2019.loc[test_2019['bq19'].str.contains('[가-힣]', na = False), ['bq19']] = int(train_2019['bq19'].mode())

In [21]:
for n, x in enumerate(train_2019['bq25']) : 
    if train_2019['bq25'][n] == '1':
        pass
    elif train_2019['bq25'][n] == '2':
        pass
    else :
        train_2019['bq25'][n] = int(train_2019['bq25'].mode())

In [22]:
for n, x in enumerate(test_2019['bq25']) : 
    if test_2019['bq25'][n] == '1':
        pass
    elif test_2019['bq25'][n] == '2':
        pass
    else :
        test_2019['bq25'][n] = int(test_2019['bq25'].mode())

### 2019년 데이터 object 컬럼 처리 

In [23]:
convert = []
for col in train_2019.select_dtypes('object'):
    if train_2019[col].nunique() < 20:
        train_2019[col] = train_2019[col].astype(float)

for col in ['bq30', 'bq31_1', 'bq31_2', 'bq31_3']:
    train_2019[col] = train_2019[col].astype(float)

In [24]:
convert = []
for col in test_2019.select_dtypes('object'):
    if test_2019[col].nunique() < 20:
        test_2019[col] = test_2019[col].astype(float)

for col in ['bq30', 'bq31_1', 'bq31_2', 'bq31_3']:
    test_2019[col] = test_2019[col].astype(float)

In [25]:
train_2019.to_csv('Data/train/1.preprocessing_train_2019.csv', index = False)
test_2019.to_csv('Data/test/1.preprocessing_test_2019.csv', index = False)